In [43]:
from gensim.models import Word2Vec

model_body = Word2Vec.load('word2vec_model')
model_total = Word2Vec.load('word2vec_model2')

In [44]:
print(model_body)
print(model_total)

Word2Vec(vocab=18254, size=100, alpha=0.025)
Word2Vec(vocab=22444, size=100, alpha=0.025)


In [21]:
from konlpy.tag import Komoran
tag = Komoran()
f = open('../Data/Preprocessed/body.txt', encoding="utf8")
sentences = f.read().strip().split('\n')
f.close()

error = []
result = []

for i, sentence in enumerate(sentences):
    try:
        result.append(tag.morphs(sentence))
    except:
        result.append(None)
        error.append(i)
        print("Error! ", i)
print(len(result))

Error!  133
Error!  810
Error!  1321
Error!  3196
Error!  3730
4368


In [22]:
f = open('../Data/Preprocessed/Morphs/body.txt', "w", encoding="utf-8")
f.write(str(result))
f.close()

In [25]:
f = open('../Data/Preprocessed/reply.txt', encoding="utf-8")
sentences = [sentence[0]['text'] for sentence in list(map(eval, f.read().strip().split("\n")))]
f.close()

error_reply = []
result_reply = []

for i, sentence in enumerate(sentences):
    try:
        result_reply.append(tag.morphs(sentence))
    except:
        result_reply.append(None)
        error_reply.append(i)
        print("Error! ", i)
print(len(result_reply))

Error!  1156

Error!  1874

Error!  2549

Error!  3047

Error!  3823

Error!  4006

4368


In [41]:
error_total = error+error_reply
error_total.sort(reverse=True)

for error_idx in error_total:
    del result[error_idx]
    del result_reply[error_idx]

4357 4357


In [42]:
f = open('../Data/Preprocessed/Morphs/body-noerror.txt', "w", encoding="utf-8")
f.write(str(result))
f.close()
f = open('../Data/Preprocessed/Morphs/reply-noerror.txt', "w", encoding="utf-8")
f.write(str(result_reply))
f.close()

In [303]:
import numpy as np

x_data, y_data = [], []
result_len = len(result)

for sentence_idx in range(result_len):
    vector_x, vector_y = [], []
    
    for word in result[sentence_idx]:
        try:
            vector_x.append(model_total.wv[word])
        except:
            pass
        
    for word in result_reply[sentence_idx]:
        try:
            vector_y.append(model_total.wv[word])
        except:
            pass
        
    x_data.append(vector_x)
    y_data.append(vector_y)
    
x_data = np.array(x_data)
y_data = np.array(y_data)

[list([array([ 0.07488272,  0.09657916,  0.3021537 , -0.4981136 ,  0.18727155,
        0.3634062 , -0.37528336, -0.18819112, -0.00857315,  0.3014509 ,
       -0.14942306,  0.10774585,  0.188709  ,  0.35541573,  0.48108   ,
       -0.35879394,  0.0073227 ,  0.19727865,  0.54227424,  0.3595264 ,
       -0.16462812,  0.12373105, -0.07307673,  0.3097458 ,  0.07789498,
        0.21857621, -0.4413762 , -0.1875228 , -0.5782256 ,  0.12298783,
       -0.3027099 , -0.04528278,  0.42467806,  0.32448646,  0.12349281,
       -0.38640493, -0.04580171, -0.04706656,  0.42221498, -0.3631304 ,
        0.0306069 ,  0.14165035,  0.25144908, -0.0032121 , -0.3062413 ,
        0.31229675,  0.3260204 , -0.07252768, -0.3462112 , -0.01135609,
        0.2969206 , -0.23630357, -0.20437314, -0.37856796,  0.51049805,
       -0.37605706,  1.0894922 , -0.03865265, -0.25050873,  0.30756387,
       -0.5490849 ,  0.0326876 , -0.2625687 ,  0.09943137,  0.17706202,
       -0.50357795, -0.19206363, -0.16215469,  0.17526202

In [96]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config)

In [310]:
import numpy as np

#for y in y_data[0]:
#    print(model_total.wv.most_similar(positive=[y], topn=1))
x_data_maxlen = 500
y_data_maxlen = 50

from keras.preprocessing import sequence
from keras import backend as K
x_data = sequence.pad_sequences(x_data, padding='post', maxlen=x_data_maxlen, dtype=K.floatx())
y_data = sequence.pad_sequences(y_data, padding='pre', maxlen=y_data_maxlen, dtype=K.floatx())

In [215]:
print(x_data.shape)
print(y_data.shape)

(4357, 500, 100)
(4357, 50, 100)


In [235]:
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, Input
from keras.layers import LSTM, SimpleRNN, GRU
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop

input_shape = x_train.shape[1:]
target_shape = y_train.shape[1:]
latent_dim = 128

encoder_inputs = Input(shape=input_shape)
#encoder_inputs = BatchNormalization()(encoder_inputs)
encoder = LSTM(latent_dim, return_sequences=True, return_state=True, kernel_initializer='zeros', recurrent_initializer='zeros')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=target_shape)
#decoder_inputs = BatchNormalization()(decoder_inputs)
decoder = LSTM(latent_dim, return_sequences=True, return_state=True, kernel_initializer='zeros', recurrent_initializer='zeros')
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(target_shape[1], activation='tanh')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)
rmsprop = RMSprop()

In [315]:
model.compile(loss='hinge', optimizer=rmsprop, metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_48 (InputLayer)           (None, 500, 100)     0                                            
__________________________________________________________________________________________________
input_49 (InputLayer)           (None, 50, 100)      0                                            
__________________________________________________________________________________________________
lstm_31 (LSTM)                  [(None, 500, 128), ( 117248      input_48[0][0]                   
__________________________________________________________________________________________________
lstm_32 (LSTM)                  [(None, 50, 128), (N 117248      input_49[0][0]                   
                                                                 lstm_31[0][1]                    
          

In [316]:
################################ 바꿔가면서 해보세요 ######################################
num_epochs = 20
mini_batch_size = 64
######################################################################################
history = model.fit([x_data, y_data], y_data, epochs=num_epochs, batch_size=mini_batch_size, verbose=1, validation_split=0.1)

Train on 3921 samples, validate on 436 samples
Epoch 1/20
3921/3921 [==============================] - 60s 15ms/step - loss: 0.9129 - acc: 0.2562 - val_loss: 0.8990 - val_acc: 0.2802
Epoch 2/20
3921/3921 [==============================] - 55s 14ms/step - loss: 0.8979 - acc: 0.2749 - val_loss: 0.8888 - val_acc: 0.2848
Epoch 3/20
3921/3921 [==============================] - 55s 14ms/step - loss: 0.8903 - acc: 0.2747 - val_loss: 0.8828 - val_acc: 0.2763
Epoch 4/20
3921/3921 [==============================] - 55s 14ms/step - loss: 0.8856 - acc: 0.2555 - val_loss: 0.8789 - val_acc: 0.2528
Epoch 5/20
3921/3921 [==============================] - 54s 14ms/step - loss: 0.8825 - acc: 0.2280 - val_loss: 0.8763 - val_acc: 0.2010
Epoch 6/20
3921/3921 [==============================] - 55s 14ms/step - loss: 0.8804 - acc: 0.1940 - val_loss: 0.8744 - val_acc: 0.1939
Epoch 7/20
3921/3921 [==============================] - 55s 14ms/step - loss: 0.8789 - acc: 0.1706 - val_loss: 0.8731 - val_acc: 0.1797
E

In [333]:
idx = 50
b = model.predict([x_data[idx:idx+1], np.zeros_like(y_data[0:1])])
for i in range(10):
    print(model_total.wv.most_similar(positive=[y_data[idx][i]], topn=1))
print()
for i in range(10):
    print(model_total.wv.most_similar(positive=[b[0][i]], topn=1))

[('받', 1.0)]
[('은', 1.0000001192092896)]
[('유치원', 1.0)]
[('이', 1.0)]
[('고', 1.0)]
[('정부', 1.0)]
[('지원', 1.0)]
[('을', 0.9999999403953552)]
[('받', 1.0)]
[('는', 0.9999999403953552)]

[('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 0.810174286365509)]
[('작살나', 0.8621266484260559)]
[('작살나', 0.890884280204773)]
[('ㄴ다지', 0.8999127149581909)]
[('ㄴ다지', 0.9036204814910889)]
[('자연현상', 0.9062573313713074)]
[('자연현상', 0.9078179597854614)]
[('자연현상', 0.9086489081382751)]
[('자연현상', 0.9091063737869263)]
[('자연현상', 0.9093635082244873)]
